In [1]:
#standard imports
import pandas as pd
import numpy as np
import os, json, math, time
from tqdm.notebook import tqdm_notebook
#import Api
import tmdbsimple as tmdb 

In [2]:
#create folder for data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_results_movie_00-01.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'results_in_progress_movie_year',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2015.json',
 'tmdb_api_results_2016.json',
 'tmdb_api_results_2017.json']

In [3]:
#loading API Credentials
with open(r"\Users\camer\.secret\timdb_api.json", "r") as f:
    login = json.load(f)
    
login.keys()

dict_keys(['api-key'])

In [4]:
#assign api key
tmdb.API_KEY = login['api-key']

In [5]:
#access tmdb api
movie_id = tmdb.Movies('imdb_id')
#creating function for movie rating
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1']=='US':
            info['certification'] = c['certification']
    return info

In [6]:
#create function to make json files
def write_json(new_data, filename): 
  
    with open(filename,'r+') as file:
       
        file_data = json.load(file)
      
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        file.seek(0)
       
        json.dump(file_data, file)

In [7]:
#adding info for years 2010-2019
YEARS_TO_GET = [2011, 2012, 2013, 2014]

In [8]:
#defining path to save in progress calls
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [9]:
#start ouloop
for YEARS in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEARS}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}], f)
    df = basics.loc[basics['startYear']==YEARS].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #inner loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc = f'Movies from {YEARS}',
                                  position = 1,
                                  leave = True):
        try: 
            temp = get_movie_with_rating(movie_id)
            write_json(temp, JSON_FILE)
            time.sleep(0.02)
        except Exception as e:
            continue
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEARS}.csv.gz", compression="gzip", index=False)


YEARS:   0%|          | 0/4 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/6756 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/7266 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/7765 [00:00<?, ?it/s]

Movies from 2014:   0%|          | 0/8013 [00:00<?, ?it/s]